In [3]:
# implementation of MFRPCA 
from scipy import linalg
import numpy as np
import torch

# O = B + M, M = E + F 
# the size of O is height * width * frame_number
# the size of B, M, E, F, X, Y is height * width * frame_number
# parameter of algorithm includes mu, rho

def cal_df(input_frame: np.ndarray, x, y, t):
    # the shape of Fh, Fv and Ft, should be 580 * 1920 * 5
    fill_up_h = np.zeros((150, 100, 10))
    Fh = input_frame[x+1, y, t] - input_frame[x, y, t]
    fill_up_v = np.zeros((150, 100, 10))
    Fv = input_frame[x, y+1, t] - input_frame[x, y, t]
    fill_up_t = np.zeros((150, 100, 10))
    Ft = input_frame[x, y, t+1] - input_frame[x, y, t]
    fill_up_h[x, y, t] = Fh.item()
    fill_up_v[x, y, t] = Fv.item()
    fill_up_t[x, y, t] = Ft.item()
    Dh = np.ones((1, 150 * 100 * 10))
    Dh = fill_up_h.reshape(1, 150 * 100 * 10)
    Dv = fill_up_v.reshape(1, 150 * 100 * 10)
    Dt = fill_up_t.reshape(1, 150 * 100 * 10)
    Dhf = np.transpose(Dh)
    #print(Dhf.shape)
    Dvf = np.transpose(Dv)
    Dtf = np.transpose(Dt)
    inner = np.stack((Dhf, Dvf, Dtf))
    Df = np.transpose(inner)
    #print(Df.shape)
    return Df, Dh, Dv, Dt, Dhf, Dvf, Dtf


In [4]:
def convert_RGB(input_img):
    if input_img.ndim == 2 or (input_img.ndim==3 and input_img.shape[2] ==1):
        # your image is in grayscale
        input_img = color.gray2rgb(input_img)
    elif input_img.ndim==3 and input_img.shape[2] == 4:
        # your image is in RGBA
        input_img = color.rgba2rgb(input_img)
    else:
        # your image is already in RGB
        assert input_img.ndim == 3 and input_img.shape[2] == 3
    return input_img

In [5]:
import cv2
from skimage import color, io

# test video segments 
def read_video_cv2(n_frames=10):
    cap = cv2.VideoCapture("./survelliance/09152008flight2tape1_7.mpg")
    all = []
    i = 0
    while cap.isOpened() and i < n_frames:
        ret, frame = cap.read()
        img = color.rgb2gray(frame)
        arr = np.array(img)
        all.append(arr)
        i += 1
    return np.array(all)

input_frame = read_video_cv2()
print(input_frame.shape)
input_seg = torch.tensor(read_video_cv2()).reshape(480, 720, 10)
input_seg_new = input_seg[0:150, 0:100, :]
print(input_seg_new.shape)

img = io.imread('./survelliance/0325.jpg')
img_res = convert_RGB(img)
print(img.shape)
#input
imgGray = color.rgb2gray(img_res)
print(imgGray.shape)

# O represents the height, width and dt frames 
# B represents the static background 
O = input_seg_new
B = np.ones((150, 100, 10)) 
# M represents the remaining parts
M = np.ones((150, 100, 10))
# E represents intrinsic foreground
E = np.ones((150, 100, 10))
# F represents dynamic background
F = np.ones((150, 100, 10))
# X and Y represent multiplier
x = np.ones((150, 100, 10))
y = np.ones((150, 100, 10))
# Z and K
Z = np.ones((3, 150, 100, 10, 1))
K = np.ones((3, 150, 100, 10, 1))

def convert_RGB(input_img):
    if input_img.ndim == 2 or (input_img.ndim==3 and input_img.shape[2] ==1):
        # your image is in grayscale
        input_img = color.gray2rgb(input_img)
    elif input_img.ndim==3 and input_img.shape[2] == 4:
        # your image is in RGBA
        input_img = color.rgba2rgb(input_img)
    else:
        # your image is already in RGB
        assert input_img.ndim == 3 and input_img.shape[2] == 3
    return input_img

def optimization(B, M, E, Df):
    res = linalg.norm(B, ord='nuc') + lambda1 * linalg.norm(M, ord=1) + lambda2 * linalg.norm(E, ord=1) + lambda3 * linalg.norm(Df, ord=0)
    return res

Df_res = cal_df(input_seg, 5, 10, 2)
print(Df_res)
# optimization(B, M, E, Df_res)

def shrinkage_operator(x, epsilon):
    # the shape of x is 1080 * 1920
    epsilon = np.zeros((150, 100))
    para1 = np.sign(x)
    print(para1.shape)
    para2 = np.maximum((np.fabs(x)-epsilon), np.zeros((150, 100)))
    print(para2.shape)
    return np.multiply(para1, para2)

(10, 480, 720)
torch.Size([150, 100, 10])
(480, 640, 3)
(480, 640)
(array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]))


In [6]:
cal_df(input_seg_new, 10, 8, 5)

(array([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]),
 array([[0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]))

In [7]:
Df_res = cal_df(input_seg, 5, 10, 2)
print(Df_res)

(array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]))


In [8]:
shrinkage_operator(input_seg_new[:, :, 0:1].reshape(150, 100), 20)

torch.Size([150, 100])
torch.Size([150, 100])


tensor([[0.0157, 0.1725, 0.4051,  ..., 0.3367, 0.3250, 0.3250],
        [0.0174, 0.1712, 0.4379,  ..., 0.3267, 0.3300, 0.3227],
        [0.0157, 0.1725, 0.4247,  ..., 0.3524, 0.3289, 0.3171],
        ...,
        [0.0140, 0.1695, 0.4800,  ..., 0.3968, 0.3990, 0.3911],
        [0.0039, 0.1569, 0.4695,  ..., 0.3794, 0.3715, 0.3715],
        [0.0000, 0.1499, 0.4573,  ..., 0.3724, 0.3488, 0.3567]],
       dtype=torch.float64)

In [9]:
svd =  np.linalg.svd(B)

In [10]:
import tensorflow as tf

mu = 0.01 
lambda1 = 10 
lambda2 = 10
lambda3 = 10 

# isotropic total variation of F. Total
def augmentated_lagrangian(O, B, E, Df, F, M, X, Y):
    opt = tf.norm(B, ord='euclidean') + lambda1 * tf.norm(M, ord=1) + lambda2 * tf.norm(E, ord=1) + lambda3 * tf.norm(Df, ord=1)
    lang1 = mu/2 * (tf.norm(np.array(O - B - M), ord='euclidean', keepdims=True) ** 2) + np.inner(x, O - B - M)
    lang2 = mu/2 * (tf.norm(M - F - E, ord='euclidean') ** 2) + np.inner(y, M - F - E)
    return opt + lang1 + lang2



In [11]:
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import gc

def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it does something you should see a number as output

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.compat.v1.Session(config=config))


# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options))


# in Df, the 3 means there are three dimensions are stacked together from fh,fv and ft
Df = np.zeros((3, 150 * 100 * 10, 1))
augmentated_lagrangian(O, B, E, Df, F, M, x, y)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
def shrinkage_operator_diagnal(x, epsilon):
    # the shape of x is 1080 * 1920
    epsilon = np.ones((150, 100, 10), dtype=np.float32)
    para1 = np.sign(x)
    # print(type(x))
    para2 = np.maximum((tf.math.abs(x)-epsilon), np.zeros((150, 100, 10), dtype=np.float32))
    return np.multiply(para1, para2)

In [ ]:
shrinkage_operator_diagnal(O, lambda1/(2 * (mu))).shape

In [ ]:
import scipy.sparse.linalg as linalg
import numpy as np 
import tensorflow as tf

input_seg_new = np.ones((150, 100, 10))
L = np.ones((10, 10))
for i in range(0, 10):
    for j in range(0, 10):
        if i == j:
            L[i][j] = 0.5

# the input of t_svd should be the size of 30 * 20 * 10 
# the result of decomposition should be U = 30 * 30 * 10 
# S = 30 * 20 * 10, V = 20 * 20 * 10
# size of L is f * f
import scipy

def t_svd(input_tube, h, w, f):
    u = np.zeros((h, h, f))
    s = np.zeros((h, w, f))
    v = np.zeros((w, w, f))
    u_hat = np.zeros((h, h, f))
    s_hat = np.zeros((h, w, f))
    v_hat = np.zeros((w, w, f))
    L_reverse = np.linalg.inv(L)
    A_hat = tf.matmul(input_tube, L)
    for it in range(0, f):
        matrix = A_hat[:, :, it]
        u, s, v = tf.linalg.svd(matrix, full_matrices = False)
        # u_hat[:, :, it] = u
        s_hat[:, :, it] = s
        v_hat[:, :, it] = v
    # [20, ] todo: the size of u_decompose is not correct. 
    u_decompose = tf.matmul(u_hat, L_reverse)
    # 30 * 20 * 10
    s_decompose = tf.matmul(s_hat, L_reverse)
    # 20 * 20 * 10
    v_decompose = tf.matmul(v_hat, L_reverse)
    return u_decompose, s_decompose, v_decompose

In [ ]:
left_high, singular_high, right_high = t_svd(input_seg_new, 150, 100, 10) 
print(left_high.shape)
print(singular_high.shape)
print(right_high.shape)

In [ ]:
import scipy
import tensorly.decomposition as decomposition
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

mu = 0.1
# 3-rank tensor multiplication
def update_B(O, M, X, mu):
    B_mid = np.zeros((O.shape[0], O.shape[1], O.shape[2]))
    B_next = np.zeros((O.shape[0], O.shape[1], O.shape[2]))
    rank = 10
    u1, d1, voft1 = t_svd(O - M + (1/mu) * X, 150, 100 ,10)
    for i in range(0, 10):
        B_mid[:, :, i] = tf.matmul(u1[:, :, i], shrinkage_operator_diagnal(d1, 1/mu)[:, :, i])
        B_next[:, :, i] = tf.matmul(B_mid[:, :, i], voft1[:, :, i])
        
    return B_next

In [ ]:
update_B(O, M, x, mu).shape

In [ ]:
print(B.shape)
def update_M(O, B_next, E, F, X, Y, mu):
    param = (O - B_next + E + F)/2 + (X - Y)/(2 * (mu))
    M_next = shrinkage_operator_diagnal(param, lambda1/(2 * (mu)))
    return M_next 

In [ ]:
update_M(O, B, E, F, x, y, mu).shape

In [ ]:
def update_E(M_next, F, Y, mu):
    param = (M_next - F + 1/(mu) * Y)
    E_next = shrinkage_operator_diagnal(param, lambda2/(mu))
    return E_next 

In [ ]:
update_E(M, F, y, mu).shape

In [ ]:
#  argmin typically used to find the smallest possible values given constraints
#  Unable to allocate 890. GiB for an array with shape (480, 720, 480, 720) and data type float64
# the video size is over large
def update_F(Df, Y, M_next, E_next, F):
    F_next = lambda3 * linalg.norm(Df, ord = 0) + np.inner(Y, M_next - E_next - F) + mu/2 * (tf.norm(M_next - E_next - F, ord="euclidean") ** 2)
    return F_next

In [ ]:
import torch

def update_multiplier(X, Y, O, B_next, M_next, E_next, F_next, mu):
    X_next = torch.tensor(x) + mu * torch.tensor(O - B_next - M_next)
    Y_next = torch.tensor(y) + mu * torch.tensor(M_next - E_next - F_next)
    return X_next, Y_next

In [ ]:
update_multiplier(x, y, O, B, M, E, F, mu)

In [ ]:
import numpy as np
import scipy

dv = 100
dh = 50
dt = 10

Dh = np.ones((dv * dh * dt, dv * dh * dt)).astype(np.int)

# the parameter of 3D fourier could change 
def three_D_fourier(input_matrix):
    fft_3D = scipy.fft.fftn(input_matrix)
    return fft_3D

In [ ]:
three_D_fourier(Dh)

In [ ]:
D = np.zeros((3 * 30 * 20 * 10, 30 * 20 * 10)) 
K_small = np.ones((3 * 30 * 20 * 10, 1))
Z_small = np.ones((3 * 30 * 20 * 10, 1))
M_small = np.ones((30, 20, 10))
E_small = np.ones((30, 20, 10))
y_small = np.ones((30, 20, 10))

# todo: the size of Q should be diagnal ----------------? 
mu = 1 
def cal_val_Q(M_next, E_next, y, D, K, Z):
    Q = np.array(M_next - E_next + y/mu).reshape(6000, 1) * mu + np.array(gamma * (np.matmul(np.transpose(D), K) + np.matmul(np.transpose(D), Z)/mu)) 
    return Q

In [ ]:
gamma = 5
# test-only
# work well 
# result shape is 480 * 720 * 10
Q_res = cal_val_Q(M_small, E_small, y_small, D, K_small, Z_small)

Q_reshape = Q_res.reshape(1, 6000)
print(Q_reshape.shape)

In [ ]:
gamma = 5
K = np.ones((3, 30 * 20 * 10, 1))
Z = np.ones((3, 30 * 20 * 10, 1))
Df = np.zeros((3, 30 * 20 * 10, 1))

'''
The shape of inner product in Ft_next_1 is 480 * 720 * 480 * 720 
returned results of above should be matching the size of each component
'''

# todo: update F does not work well ----------------------?
def update_F(K, Y, M_next, E_next, Z, Df, gamma):
    Ft_next_1 = lambda3 * tf.norm(K, ord = 1) + np.inner(Y, M_next - E_next - F) + mu/2 * (tf.norm(M_next - E_next - F, ord='euclidean') ** 2) 
    Ft_next_2 = np.inner(Z, K - Df) + gamma/2 * (tf.norm(K - Df, ord='euclidean') ** 2)
    return Ft_next_1 + Ft_next_2

In [ ]:
Dh = np.ones((1, dv * dh * dt))
Dv = np.ones((1, dv * dh * dt))
Dt = np.ones((1, dv * dh * dt))

Q_reshape.shape 
# update_F(K, y, M, E, Z, Df, gamma)

In [ ]:
# the correct size of Q should be dv * dh * dt multiply dv * dh * dt
# the size of identity matrix is dv * dh * dt multiply dv * dh * dt, 6000 * 6000
# the result of algorithm is to reshape the tensor back to 3-D tensor 30 * 20 * 10

def update_Ft(Dv, Dh, Dt, Q):
    inner_param = three_D_fourier(Q_reshape)/(mu * np.ones((1, 6000)) + gamma * (np.square(three_D_fourier(Dh)) + np.square(three_D_fourier(Dv)) + np.square(three_D_fourier(Dt))))
    ft_next = 1/three_D_fourier(inner_param)
    return ft_next.reshape(30, 20, 10)

In [ ]:
(gamma * (np.square(three_D_fourier(Dh)))).shape

In [ ]:
update_Ft(Dv, Dh, Dt, Q_reshape).shape

In [ ]:
# ph ----> dh
# pv ----> dv
# pt ----> dt
Dhf = np.ones((30 * 20 * 10, 1))
Dvf = np.ones((30 * 20 * 10, 1))
Dtf = np.ones((30 * 20 * 10, 1))
Zh = np.ones((30 * 20 * 10, 1))
Zv = np.ones((30 * 20 * 10, 1))
Zt = np.ones((30 * 20 * 10, 1))
Z = np.transpose(np.vstack((np.vstack((np.transpose(Zh),  np.transpose(Zv))), np.transpose(Zt))))
print(Z.shape)


def solution_ph(Dhf, Zh):
    return Dhf - Zh/gamma

def solution_pv(Dvf, Zv):
    return Dvf - Zv/gamma

def solution_pt(Dhf, Zt):
    return Dhf - Zt/gamma

In [ ]:
# the size of Z and Dhf, Dvf and Dtf should be same
ph = solution_ph(Dhf, Z)
pv = solution_pv(Dvf, Z)
pt = solution_pt(Dtf, Z)

In [ ]:
import numpy as np

# in Df, the 3 means there are three dimensions are stacked together from fh,fv and ft
Df = np.zeros((3, 30 * 20 * 10, 1))
# the shape of variable does not change in any defined t
# the shape of Df is same as Dvf, Dhf, Dtf

def shrinkage_operator_anisotropic(x, epsilon):
    # the shape of x is 1080 * 1920
    x = x.reshape(3, 30 * 20 * 10, 1)
    epsilon = np.zeros((3, 30 * 20 * 10, 1))
    para1 = np.sign(x)
    para2 = np.maximum((np.fabs(x)-epsilon), np.zeros((3, 30 * 20 * 10, 1)))
    return np.multiply(para1, para2)


def update_Kt_anisotropic(Df, Z, gamma):
    Df = Df.reshape(3, 30 * 20 * 10, 1)
    Z = Z.reshape(3, 30 * 20 * 10, 1)
    return shrinkage_operator_anisotropic(Df - (1/gamma * Z), lambda3/gamma)

In [ ]:
update_Kt_anisotropic(Df, Z, gamma).shape

In [ ]:
def update_Kt_isotropic(ph, pv, pt):
    epsilon = 10
    p = np.maximum(np.sqrt(np.square(ph) + np.square(pv) + np.square(pt)), epsilon)
    kt_next = np.maximum(p - lambda3/gamma, 0) * ph/p
    return kt_next

In [ ]:
Kt = update_Kt_isotropic(ph, pv, pt)

In [ ]:
print(Df.shape)
print(Kt.shape)

In [ ]:
def update_Z(Z, Kt_next, Df_next):
    Z_next = Z  + gamma * (Kt_next- Df_next)
    return Z_next

In [ ]:
update_Z(Z, Kt, Df)

In [ ]:
alpha = 0.5
rho = 1.5

def update_gamma(gamma, Kt_next, Df_next, Kt, Df):
    if tf.norm(Kt_next - Df_next, ord = 2) >= alpha * tf.norm(Kt - Df, ord = 2): 
        gamma_next = gamma * rho
    else: 
        gamma_next = gamma
    return gamma_next

In [ ]:
update_gamma(gamma, Kt, Df, Kt, Df)

In [ ]:
t = 0

def f_subproblem(Dv, Dh, Dt, Df, option:str, Q, Z, gamma, t):
    Ft_next = update_Ft(Dv, Dh, Dt, Q)
    if option == "anisotropic":
        Kt_next = update_Kt_anisotropic(Df, Z, gamma)
    elif option == "isotropic":
        ph = solution_ph(Dhf, Z)
        pv = solution_pv(Dvf, Z)
        pt = solution_pt(Dtf, Z)
        Kt_next = update_Kt_isotropic(ph, pv, pt)
    Kt_next = Kt
    Df_next = Df
    Z = update_Z(Z, Kt_next, Df_next)
    gamma = update_gamma(gamma, Kt_next, Df_next, Kt, Df)
    t = t + 1
    return Ft_next, Kt_next

In [ ]:
Df, Dh, Dv, Dt, Dhf, Dvf, Dtf = cal_df(input_seg_new, 10, 8, 5)
# F_next, K_next = f_subproblem(Dv, Dh, Dt, Df, "anisotropic", Q_reshape, Z, gamma, t)

F_next1, K_next1 = f_subproblem(Dv, Dh, Dt, Df, "anisotropic", Q_reshape, Z, gamma, t)
F_next2, K_next2 = f_subproblem(Dv, Dh, Dt, Df, "isotropic", Q_reshape, Z, gamma, t)
# result of anisotropic
print(F_next1.shape)
print(K_next1.shape)

# result of isotropic
print(F_next2.shape)
print(K_next2.shape)

In [ ]:
def tvrpca(O, B, M, E, F, X, Y, Z, K, option, Q):
    mu = 0.1
    rho = 2
    k = 0
    iter_num = 1
    t = 0
    gamma = 5
    alpha = 0.5
    while iter_num < 20:
        print("This is " + str(iter_num) + "th iteration")
        B = update_B(O, M, X, mu)
        # B_next = np.ones((30, 20, 10))
        M = update_M(O, B, E, F, X, Y, mu)
        E = update_E(M, F, Y, mu)
        F, K = f_subproblem(Dv, Dh, Dt, Df, option, Q, Z, gamma, t)
        #F = np.ones((30, 20, 10))
        X, Y = update_multiplier(X, Y, O, B, M, E, F, mu)
        k = k + 1
        mu = mu * rho
        iter_num += 1
        terminate_flag = (tf.norm(O - B - F - E, ord='euclidean')).numpy() ** 2 <= 10 ** (-7) * ((tf.norm(O, ord = 'euclidean') ** 2)).numpy()
        if terminate_flag == True:
            break
    return B, F, E, M
    

In [ ]:
tf.norm(O - B - F - E, ord='euclidean') ** 2

In [ ]:
B_res, F_res, E_res, M_res = tvrpca(O, B, M, E, F, x, y, Z, K, 'anisotropic', Q_reshape)

In [ ]:
from PIL import Image

print(B_res.shape)
im_output_sparse = Image.fromarray(F_res[:, :, 0].astype(np.uint8))
if im_output_sparse.mode != 'RGB':
    im_output_rank = im_output_sparse.convert('RGB')
im_output_rank.save("./res/tvrpca.png")
